In [1]:
!ls ./data

happiness_index.xlsx	       happiness_test_complete.csv
happiness_submit.csv	       happiness_train_abbr.csv
happiness_survey_cgss2015.pdf  happiness_train_complete.csv
happiness_test_abbr.csv        happiness_train_complete_processed.csv


In [2]:
import sys
import gc
import pandas as pd
import matplotlib
import numpy as np
import scipy as sp
from scipy import stats
from scipy.stats import norm,skew,kurtosis #for some statistics
import IPython
from IPython import display
import sklearn
import random
import time
import pickle
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns
from pylab import mpl
mpl.rcParams['font.sans-serif'] = ['FangSong'] # 指定默认字体
mpl.rcParams['axes.unicode_minus'] = False # 解决保存图像是负号'-'显示为方块的问题
%matplotlib inline
#ignore warnings
import warnings
warnings.filterwarnings('ignore')
print('import finished')

import finished


In [26]:
input_path = './data/'
file_name = 'happiness_train_complete'
train_data = pd.read_csv(input_path+file_name+'.csv',sep=',',encoding='latin-1')
# test_data = pd.read_csv(input_path+'happiness_test_complete.csv',sep=',',encoding='latin-1')
# train_data = pd.read_csv(input_path+'happiness_train_abbr.csv',sep=',',encoding='latin-1')
# test_data = train_data
# submit_example = pd.read_csv(input_path+'happiness_submit.csv',sep=',',encoding='latin-1')


print('train shape:',train_data.shape)
# print('test shape:',test_data.shape)
# print('sample shape:',submit_example.shape)

train shape: (2968, 41)


In [27]:
train_data = train_data[train_data["happiness"]!=-8].reset_index(drop=True)#注意drop=True
print('train shape:',train_data.shape)
train_data_copy = train_data.copy()
target_col = "happiness"
# target = train_data_copy[target_col]#.apply(lambda x:np.log1p(x))
# del train_data_copy[target_col]

train_shape = train_data.shape[0]
# data = pd.concat([train_data_copy,test_data],axis=0,ignore_index=True)
data = train_data
data.head()

train shape: (2968, 41)


,id,survey_type,province,city,county,survey_time,gender,birth,nationality,religion,...,family_income,family_m,family_status,house,car,marital,status_peer,status_3_before,view,inc_ability
0,8001,1,2,2,9,2015/7/24 10:30,2,1972,8,0,...,-2,5,3,1,2,3,3,1,4,2
1,8002,1,22,66,106,2015/7/12 15:38,2,1938,1,1,...,20000,1,1,2,2,7,3,3,4,2
2,8003,2,9,22,44,2015/7/5 9:36,2,1935,1,1,...,-2,1,2,1,2,7,3,1,3,4
3,8004,2,18,52,86,2015/7/19 10:10,2,1992,1,1,...,100000,2,4,1,2,3,2,1,3,2
4,8005,2,24,70,110,2015/8/3 11:41,1,1990,1,1,...,30000,3,2,1,2,1,2,2,5,2


In [28]:
#make feature
def getres1(row):
    return len([x for x in row.values if type(x)==int and x<0])

def getres2(row):
    return len([x for x in row.values if type(x)==int and x==-8])

def getres3(row):
    return len([x for x in row.values if type(x)==int and x==-1])

def getres4(row):
    return len([x for x in row.values if type(x)==int and x==-2])

def getres5(row):
    return len([x for x in row.values if type(x)==int and x==-3])
data['neg1'] = data[data.columns].apply(lambda row:getres1(row),axis=1)
data.drop(data.loc[data['neg1']>12].index.tolist(), inplace = True)
del data['neg1']

In [29]:
data.loc[data['religion']<0,'religion'] = 1
data.loc[data['religion_freq']<0,'religion_freq'] = 1
data.loc[data['edu']<0,'edu'] = 0 #TODO
data.loc[data['edu_status']<0,'edu_status'] = 4
data.loc[data['social_neighbor']<0,'social_neighbor'] = 0

# data.loc[data['income']<0,'income'] = 0
# data.loc[data['s_income']<0,'s_income']= 0
# data.loc[data['family_income']<=0,'family_income']=0
# data.loc[data['inc_exp']<=0,'inc_exp']= 0

#对体重修正
data.loc[(data['weight_jin']<=80)&(data['height_cm']>=160),'weight_jin']= data['weight_jin']*2
data.loc[data['weight_jin']<=60,'weight_jin']= data['weight_jin']*2

data.loc[data['equity']<0,'equity'] = 0 #TODO

#TODO
data.loc[data['class']<0,'class'] = 0
data.loc[data['class_10_after']<0,'class_10_after'] = 0
data.loc[data['class_10_before']<0,'class_10_before'] = 0
data.loc[data['class_14']<0,'class_14'] = 0
data.loc[data['neighbor_familiarity']<0,'neighbor_familiarity'] = 0

data.loc[data['family_m']<0,'family_m'] = 1
data.loc[data['health']<0,'health'] = 4
data.loc[data['health_problem']<0,'health_problem'] = 0
data.loc[data['nationality']<0,'nationality'] = 8

data.loc[data['inc_ability']<0,'inc_ability'] = np.nan #TODO

data.loc[data['status_peer']<0,'status_peer'] = 2
data.loc[data['status_3_before']<0,'status_3_before'] = 2
#部分特征处理
list_index = data.loc[data['edu_yr']<0,'edu_yr'].index.tolist()
data.loc[list_index, 'edu_yr'] = data.loc[list_index, 'birth'] + data.loc[list_index, 'edu'] + 10 # ADD
data.loc[data['work_status']<0,'work_status'] = 9
list_index = data.loc[data['work_yr']<0].index.tolist()
data.loc[list_index, 'work_yr'] = 2015 - data.loc[list_index, 'edu_yr'] # ADD
# data.loc[list_index, 'work_yr'] = 2015 - (data.loc[list_index, 'birth'] + data.loc[list_index, 'edu'] + 10)
list_tmp = data.loc[(data['income']==0) & (data['work_exper']==1) & (data['work_type']==-8)].index.tolist()
data.loc[list_tmp, 'work_exper'] = 6
data.loc[list_tmp, 'work_yr'] = np.NaN
data.loc[list_tmp, 'work_status'] = np.NaN
data.loc[list_tmp, 'work_type'] = np.NaN
data.loc[list_tmp, 'work_manage'] = np.NaN
data.loc[data['work_type']<0,'work_type'] = 1
data.loc[data['work_manage']<0,'work_manage'] = 2
data.loc[data['car']<0,'car'] = 2

data.loc[data['view']<0,'view'] = 2 #TODO

data['survey_time'] = pd.to_datetime(data['survey_time'])

data.loc[data['marital_1st']<0,'marital_1st']= np.nan
data.loc[data['marital_now']<0,'marital_now']= np.nan
data.loc[data['join_party']<0,'join_party']= np.nan

for i in range(1,12+1):
    data.loc[data['leisure_'+str(i)]<0,'leisure_'+str(i)] = 6
for i in range(1,9+1):
    data.loc[data['public_service_'+str(i)]<0,'public_service_'+str(i)] = data['public_service_'+str(i)].dropna().mode().values
for i in range(1,13+1):
    data.loc[data['trust_'+str(i)]<0,'trust_'+str(i)] = data['trust_'+str(i)].dropna().mode().values

In [30]:
data.to_csv(input_path+file_name+'_processed.csv',sep=',',encoding='latin-1')

In [20]:
#####################################################################################################
#
# make feature
#
######################################################################################################

# #调查问卷有效程度
# data['neg1'] = data[data.columns].apply(lambda row:getres1(row),axis=1)
# data.loc[data['neg1']>20,'neg1'] = 20  #平滑处理

# data['neg2'] = data[data.columns].apply(lambda row:getres2(row),axis=1)
# data['neg3'] = data[data.columns].apply(lambda row:getres3(row),axis=1)
# data['neg4'] = data[data.columns].apply(lambda row:getres4(row),axis=1)
# data['neg5'] = data[data.columns].apply(lambda row:getres5(row),axis=1)

#毕业年龄
data['edubir'] = data['edu_yr'] - data['birth'] 
#调查年龄
data['survey_age'] = 2015-data['birth']
#毕业后调查年数差
data['survey_edu_age'] = 2015-data['edu_yr']

#调查月、时
data['survey_month'] = data['survey_time'].dt.month
data['survey_hour']= data['survey_time'].dt.hour

#第一次结婚年龄
data['marital_1stbir'] = data['marital_1st'] - data['birth'] 
#最近结婚年龄
data['marital_nowtbir'] = data['marital_now'] - data['birth'] 
#是否再婚
data['mar'] = data['marital_nowtbir'] - data['marital_1stbir']
#配偶年龄
data['marital_sbir'] = data['marital_now']-data['s_birth']
#配偶年龄差
data['age_'] = data['marital_nowtbir'] - data['marital_sbir'] 

#收入比
data['income/s_income'] = data['income']/(data['s_income']+1)
data['income+s_income'] = data['income']+(data['s_income']+1)
data['income/family_income'] = data['income']/(data['family_income']+1)
data['all_income/family_income'] = (data['income']+data['s_income'])/(data['family_income']+1)
data['income/inc_exp'] = data['income']/(data['inc_exp']+1)
data['family_income/m'] = data['family_income']/(data['family_m']+0.01)

#收入/面积比
data['income/floor_area'] = data['income']/(data['floor_area']+0.01)
data['all_income/floor_area'] = (data['income']+data['s_income'])/(data['floor_area']+0.01)
data['family_income/floor_area'] = data['family_income']/(data['floor_area']+0.01)

data['income/m'] = data['floor_area']/(data['family_m']+0.01)

#class
data['class_10_diff'] = (data['class_10_after'] - data['class_10_before'])
data['class_diff'] = data['class'] - data['class_10_before']
data['class_14_diff'] = data['class'] - data['class_14']


#province mean
data['province_income_mean'] = data.groupby(['province'])['income'].transform('mean').values
data['province_family_income_mean'] = data.groupby(['province'])['family_income'].transform('mean').values
data['province_equity_mean'] = data.groupby(['province'])['equity'].transform('mean').values
data['province_depression_mean'] = data.groupby(['province'])['depression'].transform('mean').values
data['province_floor_area_mean'] = data.groupby(['province'])['floor_area'].transform('mean').values

#city   mean
data['city_income_mean'] = data.groupby(['city'])['income'].transform('mean').values
data['city_family_income_mean'] = data.groupby(['city'])['family_income'].transform('mean').values
data['city_equity_mean'] = data.groupby(['city'])['equity'].transform('mean').values
data['city_depression_mean'] = data.groupby(['city'])['depression'].transform('mean').values
data['city_floor_area_mean'] = data.groupby(['city'])['floor_area'].transform('mean').values


#county  mean
data['county_income_mean'] = data.groupby(['county'])['income'].transform('mean').values
data['county_family_income_mean'] = data.groupby(['county'])['family_income'].transform('mean').values
data['county_equity_mean'] = data.groupby(['county'])['equity'].transform('mean').values
data['county_depression_mean'] = data.groupby(['county'])['depression'].transform('mean').values
data['county_floor_area_mean'] = data.groupby(['county'])['floor_area'].transform('mean').values

#ratio 相比同个城市
data['income/county'] = data['income']/(data['county_income_mean']+1)                                      
data['family_income/county'] = data['family_income']/(data['county_family_income_mean']+1)   
data['equity/county'] = data['equity']/(data['county_equity_mean']+1)       
data['depression/county'] = data['depression']/(data['county_depression_mean']+1)                                                
data['floor_area/county'] = data['floor_area']/(data['county_floor_area_mean']+1)   

#age   mean
data['age_income_mean'] = data.groupby(['survey_age'])['income'].transform('mean').values
data['age_family_income_mean'] = data.groupby(['survey_age'])['family_income'].transform('mean').values
data['age_equity_mean'] = data.groupby(['survey_age'])['equity'].transform('mean').values
data['age_depression_mean'] = data.groupby(['survey_age'])['depression'].transform('mean').values
data['age_floor_area_mean'] = data.groupby(['survey_age'])['floor_area'].transform('mean').values
data['age_health_mean'] = data.groupby(['survey_age','gender'])['health'].transform('mean').values
data['age_edu_mean'] = data.groupby(['survey_age','gender'])['edu'].transform('mean').values


#age/gender   mean
data['age_income_mean'] = data.groupby(['survey_age','gender'])['income'].transform('mean').values
data['age_family_income_mean'] = data.groupby(['survey_age','gender'])['family_income'].transform('mean').values
data['age_equity_mean'] = data.groupby(['survey_age','gender'])['equity'].transform('mean').values
data['age_depression_mean'] = data.groupby(['survey_age','gender'])['depression'].transform('mean').values
data['age_floor_area_mean'] = data.groupby(['survey_age','gender'])['floor_area'].transform('mean').values
# data['age_BMI_mean'] = data.groupby(['survey_age','gender'])['BMI'].transform('mean').values
data['age_gender_health_mean'] = data.groupby(['survey_age','gender'])['health'].transform('mean').values


#class   mean
data['city_class_income_mean'] = data.groupby(['class'])['income'].transform('mean').values
data['city_class_family_income_mean'] = data.groupby(['class'])['family_income'].transform('mean').values
data['city_class_equity_mean'] = data.groupby(['class'])['equity'].transform('mean').values
data['city_class_depression_mean'] = data.groupby(['class'])['depression'].transform('mean').values
data['city_class_floor_area_mean'] = data.groupby(['class'])['floor_area'].transform('mean').values

#悠闲指数
leisure_fea_lis = ['leisure_'+str(i) for i in range(1,13)]
data['leisure_sum'] = data[leisure_fea_lis].sum() #skew

#满意指数
public_service_fea_lis = ['public_service_'+str(i) for i in range(1,10)]
data['public_service_'] = data[public_service_fea_lis].sum() #skew

data['city_public_service__mean'] = data.groupby(['city'])['public_service_'].transform('mean').values
data['public_service_cit'] = data['public_service_']-data['city_public_service__mean'] 

#信任指数
trust_fea_lis = ['trust_'+str(i) for i in range(1,10)]
data['trust_'] = data[trust_fea_lis].sum() #skew

#入党时间   
data['survey_edu_age'] = 2015-data['join_party']
data['survey_edu_age'].fillna(0,inplace=True)
del data['join_party']
del data['property_other']
del data['id']